## Introduction to ```spaCy```

## Introduction to ```spaCy```

There are a number of different NLP frameworks that you're likely to encounter. The most popular and widely-used of these are:

- ```NLTK``` (Natural Language Toolkit, old-school)
- ```UDPipe``` (Neural network based, fast and light, but not super accurate)
- ```CoreNLP``` and ```stanza``` (Created by the team at Stanford; academically robust)
- ```spaCy``` production-ready, well-documented, state-of-the-art

We'll be working with ```spaCy``` in this module, primarily because it's easy and intuitive, and also scales well.

First thing we need to do is install ```spaCy``` and the language model that we want to use.

From the command line, you should first make sure to run the setup script to install requirements:

```shell 
bash setup.sh
```

## Initializing ```spaCy```

The first thing we need to do is import ```spaCy``` __and__ the language model that we want to use.

Note that, if you want to use different langauges you want to use different language models.

In [ ]:
# create a spacy NLP class
import spacy
nlp = spacy.load("en_core_web_md") # loads the entire model spacy into the variable nlp

With the model now loaded, we can begin to do some very simple NLP tasks.

Here, we create a spaCy object and assign it to the variable ```nlp```. This is the NLP pipeline that will do all our heavy lifting, using the trained model we've specified.

Below, you can see what the pipeline does with a bit of sample text. Passing text to the nlp object gives us access to a bunch of properties, including tokens (words), parts of speech, named entities, and so on. Here's we two of them, tokens and entities. These objects, in turn, have certain methods attached to them. A full outline of available methods can be found in the spaCy docs.

In this case, for all token objects, let's return the token itself (```token.text```); its part-of-speech tag (```token.pos_```); and the grammatical dependency relations between the tokens (```token.dep_```).


In [ ]:
# a single sentence example
input_string = "My name is Ross and I have family in New York."

In [ ]:
# creating a new variable called Doc object
doc = nlp(input_string)

In [ ]:
print(doc)

My name is Ross and I come from Scotland.


__Tokenize__

In [ ]:
# tokenizing text
for token in doc:
    print(token.text) # Spacy is opinonnated, it has already made assumptions of what a token should look like
    # using .text becuase, spacy gives attributes to each token (txt, speech, gramma)

My
name
is
Ross
and
I
have
family
in
New
York
.


__Trying some more attributes__

In [ ]:
# find parts-of-speech and grammatical relations
for token in doc:
    print(token.i, token.text, token.pos_, token.dep_, token.morph) # printing token index and token text. Pos is part of speech tagging. 
    # the pos numbers are categories for verbs, nouns, etc. 
    # add _ to give a human readable output.
    # token.dep means prepositions, objects, compounds. 
    # token.morph means teanse, number of person, agreement etc

0 My PRON poss Number=Sing|Person=1|Poss=Yes|PronType=Prs
1 name NOUN nsubj Number=Sing
2 is AUX ROOT Mood=Ind|Number=Sing|Person=3|Tense=Pres|VerbForm=Fin
3 Ross PROPN attr Number=Sing
4 and CCONJ cc ConjType=Cmp
5 I PRON nsubj Case=Nom|Number=Sing|Person=1|PronType=Prs
6 have VERB conj Mood=Ind|Tense=Pres|VerbForm=Fin
7 family NOUN dobj Number=Sing
8 in ADP prep 
9 New PROPN compound Number=Sing
10 York PROPN pobj Number=Sing
11 . PUNCT punct PunctType=Peri


__NER__

Extracting named entities from a ```spaCy``` doc requires an extra step, but nothing too challenging:

In [ ]:
# extracting NERs
for ent in doc.ents: # ent means entity
    print(ent.text, ent.label_)


Ross PERSON
New York GPE


__Questions:__ 

1. What range of linguistic features is available beyond what we're looking at here? 
2. Are the same range of features available for all languages? Compare e.g. English and Danish.

## Count distribution of linguistic features

__Create doc object__

In [ ]:
# load a text file
import os 
filename = os.path.join("../data/data.txt")

In [ ]:
with open(filename, "r", encoding="utf-8") as file:
    text = file.read()

In [ ]:
# create a doc object
doc = nlp(text)

In [ ]:
type(text)

str

In [ ]:

# create empty list 
entities = []
# get named entities and add to list 
for ent in doc.ents: # ent means entity
    entities.append(ent.text)


In [ ]:
print(set(entities)) # using set to find the unique entities in the list.

{'more than two decades', 'Team Jorge', 'Jorge', 'Hanan', 'Israeli', 'more than 30', 'Tal Hanan', '50-year-old', 'Guardian'}


In [ ]:
# count nr of adjectives 
adjective_count = 0

for token in doc:
    if token.pos_ == "ADJ":
        adjective_count += 1
adjective_count

13

__Relative frequency__

In [ ]:
# find the relative frequency per 10,000 words
relative_freq = (adjective_count/len(doc)) * 10000 # len(doc) = lenght of doc 

In [ ]:
round(relative_freq, 2) # converting to round number with specified number of decimal points

866.67

## Creating neater outputs using ```pandas```

At the moment, all of our output from ```spaCy``` is in the form of lists. If we want to save these, it probably makes sense to have them saved in a more transferable format, such as CSV files or JSONs.

One very easy way to do this with Python is by using the dataframe library ```pandas```.

In [ ]:
import pandas as pd

In [ ]:
# create spaCy doc
doc = nlp(input_string)

In [ ]:
# creating empty list
annotatoins = []

for token in doc: 
    annotatoins.append((token.text, token.pos_)) # appending both to the empty list. by making them to a touple.
annotatoins

[('My', 'PRON'),
 ('name', 'NOUN'),
 ('is', 'AUX'),
 ('Ross', 'PROPN'),
 ('and', 'CCONJ'),
 ('I', 'PRON'),
 ('have', 'VERB'),
 ('family', 'NOUN'),
 ('in', 'ADP'),
 ('New', 'PROPN'),
 ('York', 'PROPN'),
 ('.', 'PUNCT')]

In [ ]:
# spaCy doc to pandas dataframe
data = pd.DataFrame(annotatoins, columns=["TOKEN", "POS"])

In [ ]:
data

,TOKEN,POS
0,My,PRON
1,name,NOUN
2,is,AUX
3,Ross,PROPN
4,and,CCONJ
5,I,PRON
6,have,VERB
7,family,NOUN
8,in,ADP
9,New,PROPN


In [ ]:
# save dataframe
outpath = os.path.join("..", "output", "annotations.csv") # creating variable which works like a function for code below
data.to_csv(outpath)

## Assignment 1

Spend some time exploring and familiarizing yourself with ```spaCy``` and ```pandas```. We'll come back to them quite a lot through this semester, so it will help to have a solid handle on how they function.

When you are ready, head over to [Assignment 1](https://classroom.github.com/a/PdNi7nPv) which takes some of the skills you've learned last week and today. The task will be to count how many times certain linguistic features occur accross different documents, and to save those results in a clear and easy-to-understand way.